# **Experimental Tests Guide for the Solar Array Simulator of the IPTC Project**

The following tests aim to evaluate the correct functionality of:

* The developed model to emulate the performance of solar arrays of different dimensions, and different conditions of irradiance, temperature, and incidence angle.
* The developed interactive control panel where the user can introduce different input parameters to remotely control a programable power supply and electronic load.
* The developed algorithm to control said programable power supply and electronic load to emulate a solar array.
* The devoloped database where the monitored parameters are stored.



First, run the following code to mount the Google Drive, so that the .CSV files that contain the required data sets for the tests can be imported:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Test #1: Track the MPP in a range of irradiance, temperature and incidence angles**

---







* This three tests aim to measure how accurately can the power supply emulate the tracking of the MPP (maximum power point) of a solar array when the levels of irradiance, temperature and incidence angle progressively increase.
* Data sets are previously generated, so that the voltage, current, power and resistance at the MPP are known for each level of irradiance, temperature and incidence angle that is evaluated. These data sets are automatically imported during the code execution, so it is not required to manually upload them.
* Said datasets are located in this [Google Drive folder](https://drive.google.com/drive/folders/14cnX26U2cmfnXcjAam81IaHVWfK7NiEt?usp=sharing), so if it is needed to modify or replace any of the data sets, it has to be done directly on that folder. Avoid changing the .CSV files' names, as that would cause conflicts in the code execution.
* The irradiance and temperature tests are carried out with a 2x3 array and an incidence angle of 54.7° on each face, which produces the maximum power generation for a 1U CubeSat \[[1](https://pressbooks-dev.oer.hawaii.edu/epet302/chapter/5-5-power-generation/#:~:text=If%20there%20are%20two%20panels,power%20of%20a%20single%20panel.)\].
* The incidence angle test is carried out with a 2x1 array, which emulates a typical configuration on one face of a 1U CubeSat, having two solar panels in series.
* Verify that the test is indeed working by checking that the input variables are changing every 30 seconds in the [control panel](https://docs.google.com/spreadsheets/d/1UBY_h6nkw9hx8eLrjqnekZ5ClKrTYQHTtDj0E-Dv7Ik/edit?usp=sharing), and by checking how new data logs are being added in the [database](https://docs.google.com/spreadsheets/d/1APg_aHuZdDOMjJTQf6wpr_L8itCIcCIbvE0jGAKACTk/edit?usp=sharing) every 5 seconds.  

**Irradiance Test**

* This test changes the level of irradiance, and its corresponding value of voltage, every 30 seconds.
* Goes through the irradiance range of 100 W/m^2 to 1375 W/m^2, doing increments of 25 W/m^2

**Temperature Test**

* This test changes the level of temperature, and its corresponding value of voltage, every 30 seconds.
* Goes through the temperature range of -80 °C to 80 °C, doing increments of 5 °C

**Incidence Angle Test**

* This test changes the levels of incidence angle, and its corresponding value of voltage, every 30 seconds.
* Goes through the incidence angle range of 0 ° to 87.5 °, doing increments of 2.5 °

**Step 1**: Run the following script, which contains all the functions and libraries needed for these tests.

In [ ]:
import time
import requests
import csv

def import_inputs(file_path):
  # Initialize empty arrays to store the values
  parameter_arr = []
  V_arr = []
  I_arr = []

  # Open the CSV file for reading
  with open(file_path, 'r') as csvfile:
      # Create a CSV reader
      csvreader = csv.reader(csvfile)

      # Skip the header row if it exists
      next(csvreader, None)

      # Iterate through the rows and extract values
      for row in csvreader:
          # Assuming the first column is G and the second column is V
          parameter = float(row[0])
          V = float(row[1])
          I = float(row[2])

          # Append the values to the respective arrays
          parameter_arr.append(parameter)
          V_arr.append(V)
          I_arr.append(I)

  print(parameter_arr)
  print(V_arr)
  print(I_arr)

  return parameter_arr, V_arr, I_arr

def set_G(PV, G_arr, V_arr, I_arr, N_s, N_p, Ang_Deg, G, T_c, base_url, CV, sampling_time):
  # Set initial inputs
  url = base_url + "D6=" + str(1) + "&D7=" + str(1) + "&D8=" + str(PV) + "&D9=" + str(N_s)
  url += "&D10=" + str(N_p) + "&D11=" + str(G) + "&D12=" + str(T_c) + "&D13=" + str(Ang_Deg) + "&D14=" + str(CV)
  response = requests.get(url)
  if response.status_code == 200:
      print("The initial inputs were set")

  if CV == 0:
    # Iterate through the array and load the URL every 5 seconds
    for i, value in enumerate(V_arr):
        url = base_url + "D11=" + str(G_arr[i]) + "&D15=" + str(V_arr[i])
        response = requests.get(url)

        if response.status_code == 200:
            print(f"G = {G_arr[i]}     V = {V_arr[i]}")

        # Wait for 30 seconds before the next iteration
        time.sleep(30)

  elif CV == 1:
    # Iterate through the array and load the URL every 5 seconds
    for i, value in enumerate(I_arr):
        url = base_url + "D11=" + str(G_arr[i]) + "&D15=" + str(I_arr[i])
        response = requests.get(url)

        if response.status_code == 200:
            print(f"G = {G_arr[i]}     I = {I_arr[i]}")

        # Wait for 30 seconds before the next iteration
        time.sleep(sampling_time)

def set_T(PV, T_arr, V_arr, I_arr, N_s, N_p, Ang_Deg, G, T_c, base_url, CV, sampling_time):
  # Set initial inputs
  url = base_url + "D6=" + str(1) + "&D7=" + str(1) + "&D8=" + str(PV) + "&D9=" + str(N_s)
  url += "&D10=" + str(N_p) + "&D11=" + str(G) + "&D12=" + str(T_c) + "&D13=" + str(Ang_Deg) + "&D14=" + str(CV)
  response = requests.get(url)
  if response.status_code == 200:
      print("The initial inputs were set")

  if CV == 0:
    # Iterate through the array and load the URL every 5 seconds
    for i, value in enumerate(V_arr):
        url = base_url + "D12=" + str(T_arr[i]) + "&D15=" + str(V_arr[i])
        response = requests.get(url)

        if response.status_code == 200:
            print(f"T = {T_arr[i]}     V = {V_arr[i]}")

        # Wait for 30 seconds before the next iteration
        time.sleep(30)

  elif CV == 1:
    # Iterate through the array and load the URL every 5 seconds
    for i, value in enumerate(I_arr):
        url = base_url + "D12=" + str(T_arr[i]) + "&D15=" + str(I_arr[i])
        response = requests.get(url)

        if response.status_code == 200:
            print(f"T = {T_arr[i]}     I = {I_arr[i]}")

        # Wait for 30 seconds before the next iteration
        time.sleep(sampling_time)

def set_Ang(PV, Ang_arr, V_arr, I_arr, N_s, N_p, Ang_Deg, G, T_c, base_url, CV, sampling_time):
  # Set initial inputs
  url = base_url + "D6=" + str(1) + "&D7=" + str(1) + "&D8=" + str(PV) + "&D9=" + str(N_s)
  url += "&D10=" + str(N_p) + "&D11=" + str(G) + "&D12=" + str(T_c) + "&D13=" + str(Ang_Deg) + "&D14=" + str(CV)
  response = requests.get(url)
  if response.status_code == 200:
      print("The initial inputs were set")

  if CV == 0:
    # Iterate through the array and load the URL every 5 seconds
    for i, value in enumerate(V_arr):
        url = base_url + "D13=" + str(Ang_arr[i]) + "&D15=" + str(V_arr[i])
        response = requests.get(url)

        if response.status_code == 200:
            print(f"Ang = {Ang_arr[i]}     V = {V_arr[i]}")

        # Wait for 30 seconds before the next iteration
        time.sleep(30)

  elif CV == 1:
    for i, value in enumerate(I_arr):
        url = base_url + "D13=" + str(Ang_arr[i]) + "&D15=" + str(I_arr[i])
        response = requests.get(url)

        if response.status_code == 200:
            print(f"Ang = {Ang_arr[i]}     I = {I_arr[i]}")

        # Wait for 30 seconds before the next iteration
        time.sleep(sampling_time)

def set_initial_conditions(PV, test):
  if test == "G" or test == "T":
    N_s = 2             # Number of rows in the solar array (Ns)
    N_p = 3             # Number of columns in the solar array (Np)
    Ang_Deg = 54.7      # Incidence angle

  elif test == "Ang":
    N_s = 2             # Number of rows in the solar array (Ns)
    N_p = 1             # Number of columns in the solar array (Np)
    Ang_Deg = 0        # Incidence angle

  if PV == 1:
    G = 1367            # Irradiance
    T_c = 28            # Temperature of the cell

  elif PV == 2 or PV == 3:
    G = 1367            # Irradiance
    T_c = 25            # Temperature of the cell

  elif PV == 4:
    G = 1353            # Irradiance
    T_c = 28            # Temperature of the cell

  print(N_s, N_p, Ang_Deg, G, T_c)

  return N_s, N_p, Ang_Deg, G, T_c

def turn_off_equipment(base_url):
  url = base_url + "D14=0&D15=0"
  response = requests.get(url)
  if response.status_code == 200:
    print("Voltage set to zero")

  url = base_url + "D7=0"
  response = requests.get(url)
  if response.status_code == 200:
    print("The equipment was turned off")
  time.sleep(1)

def irradiance_test(PV, file_path_G, base_url, sampling_time):
  print("Starting irradiance test...")
  N_s, N_p, Ang_Deg, G, T_c = set_initial_conditions(PV, "G")
  G_arr, V_arr, I_arr = import_inputs(file_path_G)
  set_G(PV, G_arr, V_arr, I_arr, N_s, N_p, Ang_Deg, G, T_c, base_url, 0, sampling_time)
  #set_G(PV, G_arr, V_arr, I_arr, base_url, 1, sampling_time)
  print("Irradiance test finished!")

def temperature_test(PV, file_path_T, base_url, sampling_time):
  print("Starting temperature test...")
  N_s, N_p, Ang_Deg, G, T_c = set_initial_conditions(PV, "T")
  T_arr, V_arr, I_arr = import_inputs(file_path_T)
  set_T(PV, T_arr, V_arr, I_arr, N_s, N_p, Ang_Deg, G, T_c, base_url, 0, sampling_time)
  #set_T(PV, T_arr, V_arr, I_arr, base_url, 1, sampling_time)
  print("Temperature test finished!")

def incidence_angle_test(PV, file_path_Ang, base_url, sampling_time):
  print("Starting incidence angle test...")
  N_s, N_p, Ang_Deg, G, T_c = set_initial_conditions(PV, "Ang")
  Ang_arr, V_arr, I_arr = import_inputs(file_path_Ang)
  set_Ang(PV, Ang_arr, V_arr, I_arr, N_s, N_p, Ang_Deg, G, T_c,  base_url, 0, sampling_time)
  #set_Ang(PV, Ang_arr, V_arr, I_arr, base_url, 1, sampling_time)
  print("Incidence angle test finished!")

def run_all_conditions_tests(PV, G, T, Ang, base_url, sampling_time):
  base_file_path = '/content/drive/MyDrive/Datasets_for_Tests/Diff_Conditions_Test/'
  if PV == 1:
    file_path_G_AZUR_3G30A = base_file_path + "AZUR_3G30A_MPP_for_a_given_range_of_G.csv"
    file_path_T_AZUR_3G30A = base_file_path + "AZUR_3G30A_MPP_for_a_given_range_of_T.csv"
    file_path_Ang_AZUR_3G30A = base_file_path + "AZUR_3G30A_MPP_for_a_given_range_of_Ang.csv"
    if G == 1:
      irradiance_test(PV, file_path_G_AZUR_3G30A, base_url, sampling_time)
      turn_off_equipment(base_url)
    if T == 1:
      temperature_test(PV, file_path_T_AZUR_3G30A, base_url, sampling_time)
      turn_off_equipment(base_url)
    if Ang == 1:
      incidence_angle_test(PV, file_path_Ang_AZUR_3G30A, base_url, sampling_time)
      turn_off_equipment(base_url)
  elif PV == 2:
    file_path_G_AZUR_4G32C = base_file_path + "AZUR_4G32C_MPP_for_a_given_range_of_G.csv"
    file_path_T_AZUR_4G32C = base_file_path + "AZUR_4G32C_MPP_for_a_given_range_of_T.csv"
    file_path_Ang_AZUR_4G32C = base_file_path + "AZUR_4G32C_MPP_for_a_given_range_of_Ang.csv"
    if G ==1:
      irradiance_test(PV, file_path_G_AZUR_4G32C, base_url, sampling_time)
      turn_off_equipment(base_url)
    if T == 1:
      temperature_test(PV, file_path_T_AZUR_4G32C, base_url, sampling_time)
      turn_off_equipment(base_url)
    if Ang == 1:
      incidence_angle_test(PV, file_path_Ang_AZUR_4G32C, base_url, sampling_time)
      turn_off_equipment(base_url)

  elif PV == 3:
    file_path_G_CESI_CTJ30 = base_file_path + "CESI_CTJ30_MPP_for_a_given_range_of_G.csv"
    file_path_T_CESI_CTJ30 = base_file_path + "CESI_CTJ30_MPP_for_a_given_range_of_T.csv"
    file_path_Ang_CESI_CTJ30 = base_file_path + "CESI_CTJ30_MPP_for_a_given_range_of_Ang.csv"
    if G == 1:
      irradiance_test(PV, file_path_G_CESI_CTJ30, base_url, sampling_time)
      turn_off_equipment(base_url)
    if T == 1:
      temperature_test(PV, file_path_T_CESI_CTJ30, base_url, sampling_time)
      turn_off_equipment(base_url)
    if Ang == 1:
      incidence_angle_test(PV, file_path_Ang_CESI_CTJ30, base_url, sampling_time)
      turn_off_equipment(base_url)

  elif PV == 4:
    file_path_G_Spectrolab_UTJ = base_file_path + "Spectrolab_UTJ_MPP_for_a_given_range_of_G.csv"
    file_path_T_Spectrolab_UTJ = base_file_path + "Spectrolab_UTJ_MPP_for_a_given_range_of_T.csv"
    file_path_Ang_Spectrolab_UTJ = base_file_path + "Spectrolab_UTJ_MPP_for_a_given_range_of_Ang.csv"
    if G == 1:
      irradiance_test(PV, file_path_G_Spectrolab_UTJ, base_url, sampling_time)
      turn_off_equipment(base_url)
    if T == 1:
      temperature_test(PV, file_path_T_Spectrolab_UTJ, base_url, sampling_time)
      turn_off_equipment(base_url)
    if Ang == 1:
      incidence_angle_test(PV, file_path_Ang_Spectrolab_UTJ, base_url, sampling_time)
      turn_off_equipment(base_url)

  elif PV == 5:
    file_path_G_New_Panel = "New_Panel_MPP_for_a_given_range_of_G.csv"
    file_path_T_New_Panel = "New_Panel_MPP_for_a_given_range_of_T.csv"
    file_path_Ang_New_Panel = "New_Panel_MPP_for_a_given_range_of_Ang.csv"
    if G == 1:
      irradiance_test(PV, file_path_G_New_Panel, base_url, sampling_time)
      turn_off_equipment(base_url)
    if T == 1:
      temperature_test(PV, file_path_T_New_Panel, base_url, sampling_time)
      turn_off_equipment(base_url)
    if Ang == 1:
      incidence_angle_test(PV, file_path_Ang_New_Panel, base_url, sampling_time)
      turn_off_equipment(base_url)

**Step 2:** Set which tests you wish to run and then run the following script.


The function **run_all_conditions_tests()** takes 5 arguments, set them according to your needs:
* **PV**: solar panel identification number (1 = AZUR 3G30A, 2 = AZUR 4G32C, 3 = CESI CTJ30, 4 = Spectrolab UTJ, 5 = New Panel).
* **G, T_c; Ang**: boolean variable to indicate if you want to run the irradiance. temperature, and incidence angle tests, respectively (set it to **1** if you want to test it, set it to **0** if not).
* **base_url**: the base URL of the Google Sheets data base. If the Google Apps Script linked to these sheet is modified, this URL has to be updated.
* **sampling_time**: new conditions are sent to the control panel according to this value, in seconds. By default, it's defined to 30 seconds, but you can change it according to your needs.

**Note**: the function is called once for each solar panel model, if there's a panel you don't wish to test, comment its respective line in the script.

In [ ]:
sampling_time = 30 # in seconds

base_url = "https://script.google.com/macros/s/AKfycbzPEco1aFMXpkTYXp6zrzSQGMNOIWgrQhZDs9_kuemn_6uvIfw7OYaTVTA48EesBWHX/exec?"

# Select the tests you wish to run:

#                        PV   G    T    Ang
run_all_conditions_tests(1,   1,   1,   1,   base_url,   sampling_time) # AZUR 3G30A
run_all_conditions_tests(2,   1,   1,   1,   base_url,   sampling_time) # AZUR 4G32C
run_all_conditions_tests(3,   1,   1,   1,   base_url,   sampling_time) # CESI CTJ30
run_all_conditions_tests(4,   1,   1,   1,   base_url,   sampling_time) # Spectrolab UTJ
#run_all_conditions_tests(5,   1,   1,   1,   base_url,   sampling_time) # New Panel

print("All tests finished!")

# **Test #2: Characteristic curves for different array sizes**

---







* This tests aim to measure how accurately the power supply emulates the characteristic curves of solar arrays of different dimensions.
* Data sets previously generated, so that the voltage, current, power and resistance at different points of interest are know beforehand. These data sets are automatically imported during the code execution, so it is not required to manually upload them.
* Said datasets are located in this [Google Drive folder](https://drive.google.com/drive/folders/1CM1ZJKUG1mT_reWbXoDuoGXlNbkOo2zI?usp=sharing), so if it is needed to modify or replace any of the data sets, it has to be done directly on that folder. Avoid changing the .CSV files' names, as that would cause conflicts in the code execution.
* The tests evaluate N_s x N_p array sizes of 1x1, 1x2, 2x1, 2x3, 1x6, 6x1, at the standard conditions of temperature and irradiance of each panel, with a 0° incidence angle, and the control variable is voltage.
* Verify that the test is indeed working by checking that the input variables are changing every 30 seconds in the [control panel](https://docs.google.com/spreadsheets/d/1UBY_h6nkw9hx8eLrjqnekZ5ClKrTYQHTtDj0E-Dv7Ik/edit?usp=sharing), and by checking how new data logs are being periodically added in the [database](https://docs.google.com/spreadsheets/d/1APg_aHuZdDOMjJTQf6wpr_L8itCIcCIbvE0jGAKACTk/edit?usp=sharing).

**Step 1**: Run the following script, which contains all the functions and libraries needed for this tests.

In [ ]:
import csv
import time
import requests

def import_dataset(file_path):
  V_arr = []
  # Open the CSV file for reading
  with open(file_path, 'r') as csvfile:
      # Create a CSV reader
      csvreader = csv.reader(csvfile)

      # Skip the header row if it exists
      next(csvreader, None)

      # Iterate through the rows and extract values
      for row in csvreader:
          V = float(row[0])

          # Append the values to the respective arrays
          V_arr.append(V)
  return V_arr

def send_to_control_panel(PV, V_arr, G, T_c, N_s, N_p, start_index, base_url, sampling_time):
  # Set initial inputs
  Update = 1
  Start = 1           # 1 = Start, 0 = Stop
  Ang_Deg = 0         # Incidence angle
  CV = 0              # Control variable. V = 0, I = 1

  url = base_url + "D6=" + str(Update) + "&D7=" + str(Start) + "&D8=" + str(PV) + "&D9=" + str(N_s)
  url += "&D10=" + str(N_p) + "&D11=" + str(G) + "&D12=" + str(T_c) + "&D13=" + str(Ang_Deg) + "&D14=" + str(CV)
  response = requests.get(url)
  if response.status_code == 200:
      print("The initial inputs were set")

  # Iterate through the array and load the URL every 5 seconds
  for i in range(start_index, len(V_arr)):
      url = base_url + "D15=" + str(V_arr[i])
      response = requests.get(url)

      if response.status_code == 200:
          print(f"V = {V_arr[i]}")

      # Wait for X seconds before the next iteration
      time.sleep(sampling_time)

def test_array(PV, PV_name,  G, T_c, N_s, N_p, start_index, sampling_time):
  array_size_str = str(N_s) + "x" + str(N_p)
  base_file_path = '/content/drive/MyDrive/Datasets_for_Tests/Diff_Array_Sizes_Test/'
  file_path = base_file_path + PV_name + "_datasets_for_array_" + array_size_str + ".csv"
  print("Testing the " + array_size_str + " array...")
  V_arr = import_dataset(file_path)
  send_to_control_panel(PV, V_arr,  G, T_c, N_s, N_p, start_index, base_url, sampling_time)
  print("Test for the " + array_size_str + " array finished!")

def turn_off_equipment(base_url):
  url = base_url + "D14=0&D15=0"
  response = requests.get(url)
  if response.status_code == 200:
    print("Voltage set to zero")

  url = base_url + "D7=0"
  response = requests.get(url)
  if response.status_code == 200:
    print("The equipment was turned off")
  time.sleep(1)

def load_panel_data(PV):
  # Inputs for AZUR 3G30A
  if PV == 1:              # Solar panel
    PV_name = "AZUR_3G30A"
    G = 1367            # Irradiance
    T_c = 28            # Temperature of the cell

  # Inputs for AZUR 4G32C
  elif PV == 2:
    PV_name = "AZUR_4G32C"
    G = 1367            # Irradiance
    T_c = 25            # Temperature of the cell

  # Inputs for CESI CTJ30
  elif PV == 3:
    PV_name = "CESI_CTJ30"
    G = 1367            # Irradiance
    T_c = 25            # Temperature of the cell

  # Inputs for Spectrolab UTJ
  elif PV == 4:
    PV_name = "Spectrolab_UTJ"
    G = 1353            # Irradiance
    T_c = 28            # Temperature of the cell

  # Inputs for a new panel
  elif PV == 5:
    PV_name = "New_Panel"
    G = 1367            # Irradiance
    T_c = 25            # Temperature of the cell

  return PV_name, G, T_c

def run_all_arrays_tests(PV, arr_1x1, arr_1x2, arr_2x1, arr_2x3, arr_1x6, arr_6x1, base_url, sampling_time):
  PV_name, G, T_c = load_panel_data(PV)

  # 1x1 array
  if arr_1x1 == 1:
    test_array(PV, PV_name,  G, T_c, 1, 1, 0, sampling_time)
    turn_off_equipment(base_url)

  # 1x2 array
  if arr_1x2 == 1:
    test_array(PV, PV_name,  G, T_c, 1, 2, 0, sampling_time)
    turn_off_equipment(base_url)

  # 2x1 array
  if arr_2x1 == 1:
    test_array(PV, PV_name,  G, T_c, 2, 1, 0, sampling_time)
    turn_off_equipment(base_url)

  # 2x3 array
  if arr_2x3 == 1:
    test_array(PV, PV_name,  G, T_c, 2, 3, 0, sampling_time)
    turn_off_equipment(base_url)

  # 1x6 array
  if arr_1x6 == 1:
    test_array(PV, PV_name,  G, T_c, 1, 6, 0, sampling_time)
    turn_off_equipment(base_url)

  # 6x1 array
  if arr_6x1 == 1:
    test_array(PV, PV_name,  G, T_c, 6, 1, 0, sampling_time)
    turn_off_equipment(base_url)

**Step 2:** Set which tests you wish to run and then run the following script.


The function **run_all_arrays_tests()** takes 8 arguments, set them according to your needs:
* **PV**: solar panel identification number (1 = AZUR 3G30A, 2 = AZUR 4G32C, 3 = CESI CTJ30, 4 = Spectrolab UTJ, 5 = New Panel).
* **arr_1x1, arr_1x2, arr_2x1, arr_2x3, arr_1x6, arr_6x1**: boolean variable to indicate if you want to test the corresponding array dimension (set it to **1** if you want to test it, set it to **0** if not).
* **base_url**: the base URL of the Google Sheets data base. If the Google Apps Script linked to these sheet is modified, this URL has to be updated.
* **sampling_time**: new conditions are sent to the control panel according to this value, in seconds. By default, it's defined to 30 seconds, but you can change it according to your needs.

**Note**: the function is called once for each solar panel model, if there's a panel you don't wish to test, comment its respective line in the script.

In [ ]:
sampling_time = 30 # in seconds

base_url = "https://script.google.com/macros/s/AKfycbzPEco1aFMXpkTYXp6zrzSQGMNOIWgrQhZDs9_kuemn_6uvIfw7OYaTVTA48EesBWHX/exec?"

# Select the tests you wish to run:

#                    PV   1x1  1x2  2x1  2x3  1x6  6x1
run_all_arrays_tests(1,   1,   1,   1,   1,   1,   1,   base_url,   sampling_time) # AZUR 3G30A
run_all_arrays_tests(2,   1,   1,   1,   1,   1,   1,   base_url,   sampling_time) # AZUR 4G32C
run_all_arrays_tests(3,   1,   1,   1,   1,   1,   1,   base_url,   sampling_time) # CESI CTJ30
run_all_arrays_tests(4,   1,   1,   1,   1,   1,   1,   base_url,   sampling_time) # Spectrolab UTJ
#run_all_arrays_tests(5,   1,   1,   1,   1,   1,   1,   base_url,   sampling_time) # New Panel

print("Tests finished!")